In [3]:
import imagingcontrol4 as ic4

In [4]:
ic4.Library.init()

In [5]:
ic4.DeviceEnum.devices()

[DeviceInfo(model_name='DMM 37UX178-ML', serial='19814505', version='IMX178_M.MX/1895/910 USB3mx-IMX178/18')]

In [6]:
grabber = ic4.Grabber()

# Open the first available video capture device
first_device_info = ic4.DeviceEnum.devices()[0]
grabber.device_open(first_device_info)

In [7]:
grabber.device_property_map.set_value(ic4.PropId.PIXEL_FORMAT, ic4.PixelFormat.Mono16)

# Set the resolution to 640x480
grabber.device_property_map.set_value(ic4.PropId.WIDTH, 1024)
grabber.device_property_map.set_value(ic4.PropId.HEIGHT, 1024)

In [8]:
grabber.device_property_map.set_value(ic4.PropId.ACQUISITION_FRAME_RATE, 30)

In [9]:
grabber.device_property_map.set_value(ic4.PropId.EXPOSURE_AUTO, "Off")
grabber.device_property_map.set_value(ic4.PropId.EXPOSURE_TIME, 5000.0) # 单位：µs

In [11]:
a

In [19]:
a = grabber.device_property_map.get_value_float(ic4.PropId.ACQUISITION_FRAME_RATE)
a



30.0

In [14]:
a.value()

IC4Exception: (<ErrorCode.DeviceInvalid: 13>, 'ic4_prop_integer_get_value: Device has been closed')

In [8]:
sink = ic4.SnapSink()
# Setup data stream from the video capture device to the sink and start image acquisition.
grabber.stream_setup(sink, setup_option=ic4.StreamSetupOption.ACQUISITION_START)

In [11]:
try:
    # Grab a single image out of the data stream.
    image = sink.snap_single(1000)
    image_np = image.numpy_wrap()

    # Print image information.
    print(f"Received an image. ImageType: {image_np}")

    # Save the image.
    image.save_as_bmp("test.bmp")

except ic4.IC4Exception as ex:
    print(ex.message)

Received an image. ImageType: [[[44128]
  [44112]
  [44144]
  ...
  [44304]
  [44144]
  [43904]]

 [[44560]
  [43872]
  [44016]
  ...
  [44448]
  [44080]
  [45104]]

 [[44096]
  [43808]
  [43200]
  ...
  [44704]
  [45296]
  [44240]]

 ...

 [[43760]
  [43808]
  [44560]
  ...
  [43504]
  [43696]
  [43024]]

 [[45008]
  [43408]
  [44256]
  ...
  [44000]
  [43328]
  [43312]]

 [[44320]
  [43584]
  [44112]
  ...
  [43680]
  [43040]
  [43664]]]


In [1]:
import imagingcontrol4 as ic4
import numpy as np

class NumpyCaptureListener(ic4.QueueSinkListener):
    def __init__(self):
        self.latest_frame = None
    
    def sink_connected(self, sink: ic4.QueueSink, image_type: ic4.ImageType, min_buffers_required: int) -> bool:
        return True
    
    def frames_queued(self, sink: ic4.QueueSink):
        buffer = sink.pop_output_buffer()
        # 关键修复：直接使用 numpy_wrap() 无需 with
        np_array = buffer.numpy_wrap()
        self.latest_frame = np_array.copy()  # 复制数据确保安全

def capture_image_to_numpy():
    ic4.Library.init()
    
    try:
        device_list = ic4.DeviceEnum.devices()
        if not device_list:
            raise RuntimeError("未找到可用设备")
        dev_info = device_list[0]


        grabber = ic4.Grabber()
        grabber.device_open(dev_info)  # 注意原代码此处可能有拼写错误，应为 device_open
        
        listener = NumpyCaptureListener()
        sink = ic4.QueueSink(
            listener, 
            [ic4.PixelFormat.Mono16], 
            max_output_buffers=1
        )
        grabber.stream_setup(sink)
        
        print("开始采集（按Ctrl+C停止）...")
        while True:
            if listener.latest_frame is not None:
                print(f"捕获图像形状: {listener.latest_frame.shape}")
                listener.latest_frame = None
                
    except KeyboardInterrupt:
        print("用户中断，停止采集")
    finally:
        if 'grabber' in locals():
            grabber.stream_stop()
            grabber.device_close()
        ic4.Library.exit()

if __name__ == "__main__":
    capture_image_to_numpy()

开始采集（按Ctrl+C停止）...


Exception ignored in: <function Grabber.__del__ at 0x0000000008DD3430>
Traceback (most recent call last):
  File "c:\ProgramData\miniconda3\envs\qt\lib\site-packages\imagingcontrol4\grabber.py", line 98, in __del__
    Library.core.ic4_grabber_unref(self._handle)
  File "c:\ProgramData\miniconda3\envs\qt\lib\site-packages\imagingcontrol4\library.py", line 20, in core
    raise RuntimeError("Library.init was not called")
RuntimeError: Library.init was not called
Exception ignored in: <function DeviceInfo.__del__ at 0x0000000008DB2280>
Traceback (most recent call last):
  File "c:\ProgramData\miniconda3\envs\qt\lib\site-packages\imagingcontrol4\devenum.py", line 24, in __del__
    Library.core.ic4_devinfo_unref(self._handle)
  File "c:\ProgramData\miniconda3\envs\qt\lib\site-packages\imagingcontrol4\library.py", line 20, in core
    raise RuntimeError("Library.init was not called")
RuntimeError: Library.init was not called


用户中断，停止采集


In [4]:
import imagingcontrol4 as ic4
import numpy as np
import cv2

class NumpyCaptureListener(ic4.QueueSinkListener):
    def __init__(self):
        self.latest_frame = None  # 存储最新帧的NumPy数组
    
    def sink_connected(self, sink: ic4.QueueSink, image_type: ic4.ImageType, min_buffers_required: int) -> bool:
        return True  # 接受所有图像格式配置
    
    def frames_queued(self, sink: ic4.QueueSink):
        buffer = sink.pop_output_buffer()
        # 将图像数据转换为NumPy数组（假设像素格式为BGR8，与OpenCV兼容）
        np_array = buffer.numpy_wrap()
        self.latest_frame = np_array.copy()  # 必须复制数据，确保缓冲区释放后数据有效

def capture_and_display():
    ic4.Library.init()
    
    try:
        # 枚举并选择第一个可用设备
        device_list = ic4.DeviceEnum.devices()
        if not device_list:
            raise RuntimeError("未找到可用设备")
        dev_info = device_list[0]
        # print(f"正在连接设备: {dev_info.display_name}")

        # 创建设备并配置流
        grabber = ic4.Grabber()
        grabber.device_open(dev_info)
        
        # 创建监听器并配置队列Sink
        listener = NumpyCaptureListener()
        sink = ic4.QueueSink(
            listener, 
            [ic4.PixelFormat.Mono8],  # 指定BGR8格式，与OpenCV兼容
            max_output_buffers=1      # 仅保留最新帧
        )
        grabber.stream_setup(sink)
        
        # 启动采集
        print("开始采集（按 'q' 停止）...")
        while True:
            if listener.latest_frame is not None:
                # 使用OpenCV显示图像
                cv2.imshow("Camera Feed", listener.latest_frame)
                listener.latest_frame = None  # 清除已处理帧
            
            # 检测按键输入（每1毫秒检查一次）
            key = cv2.waitKey(1)
            if key == ord('q'):  # 按 'q' 退出
                break

    except Exception as e:
        print(f"错误发生: {e}")
    finally:
        # 清理资源
        if 'grabber' in locals():
            grabber.stream_stop()
            grabber.device_close()
        cv2.destroyAllWindows()  # 关闭所有OpenCV窗口
        ic4.Library.exit()

if __name__ == "__main__":
    capture_and_display()

开始采集（按 'q' 停止）...


Exception ignored in: <function Grabber.__del__ at 0x0000000008DD3430>
Traceback (most recent call last):
  File "c:\ProgramData\miniconda3\envs\qt\lib\site-packages\imagingcontrol4\grabber.py", line 98, in __del__
    Library.core.ic4_grabber_unref(self._handle)
  File "c:\ProgramData\miniconda3\envs\qt\lib\site-packages\imagingcontrol4\library.py", line 20, in core
    raise RuntimeError("Library.init was not called")
RuntimeError: Library.init was not called
Exception ignored in: <function DeviceInfo.__del__ at 0x0000000008DB2280>
Traceback (most recent call last):
  File "c:\ProgramData\miniconda3\envs\qt\lib\site-packages\imagingcontrol4\devenum.py", line 24, in __del__
    Library.core.ic4_devinfo_unref(self._handle)
  File "c:\ProgramData\miniconda3\envs\qt\lib\site-packages\imagingcontrol4\library.py", line 20, in core
    raise RuntimeError("Library.init was not called")
RuntimeError: Library.init was not called


In [3]:
import cv2

In [ ]:
image_np.shape

(1024, 1024, 1)

In [ ]:
import numpy as np